In [0]:
# Configure ADLS Gen2 access
storage_acct = "goodreadsreviews60107070"

spark.conf.set(
"fs.azure.account.key.goodreadsreviews60107070.dfs.core.windows.net",
"W68YOwummMkTfxvE8uSyeHwSn2ISU3fxF43SpgTIU/zdUDmwquZ95QpaxDJnze6PRovNww3bWamU+AStmbtZLg=="
)

# Define lakehouse paths
container   = "lakehouse"
silver_path = f"abfss://{container}@{storage_acct}.dfs.core.windows.net/processed"
gold_path   = f"abfss://{container}@{storage_acct}.dfs.core.windows.net/gold"

# load the curated Delta table directly
curated = spark.read.format("delta").table("curated_reviews")

In [0]:
# SET CORRECT DATA TYPES
from pyspark.sql import functions as F

df = curated

df = (
    df
    # Identifier columns → string
    .withColumn("review_id", F.col("review_id").cast("string"))
    .withColumn("book_id",   F.col("book_id").cast("string"))
    .withColumn("author_id", F.col("author_id").cast("string"))
    .withColumn("user_id",   F.col("user_id").cast("string"))
    
    # Numeric columns
    .withColumn("rating",  F.col("rating").cast("int"))
    .withColumn("n_votes", F.col("n_votes").cast("int"))
    
    # Text columns
    .withColumn("title",       F.col("title").cast("string"))
    .withColumn("name",        F.col("name").cast("string"))
    .withColumn("review_text", F.col("review_text").cast("string"))
)

df.printSchema()


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)



In [0]:
# FIX DATETIME PARSING FOR SPARK 3.x+
from pyspark.sql import functions as F

# Set legacy parser policy for old-style date strings
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Parse and convert "Wed Dec 15 08:57:20 -0800 2010" → yyyy-MM-dd
df = (
    df
    .withColumn(
        "date_added_parsed",
        F.to_timestamp(F.col("date_added"), "EEE MMM dd HH:mm:ss Z yyyy")
    )
    .withColumn("date_added_iso", F.date_format("date_added_parsed", "yyyy-MM-dd"))
    .withColumn("date_added_iso", F.col("date_added_iso").cast("date"))
)

df.select("date_added", "date_added_iso").show(5, truncate=False)
df.printSchema()


+------------------------------+--------------+
|date_added                    |date_added_iso|
+------------------------------+--------------+
|Wed Dec 15 08:57:20 -0800 2010|2010-12-15    |
|Wed Nov 03 12:59:13 -0700 2010|2010-11-03    |
|Tue Feb 28 17:17:01 -0800 2017|2017-03-01    |
|Fri Jul 27 14:06:28 -0700 2012|2012-07-27    |
|Mon Jan 30 17:59:47 -0800 2017|2017-01-31    |
+------------------------------+--------------+
only showing top 5 rows
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_added_parsed: timestamp (nullable = true)
 |-- date_added_iso: date (nullable = true)



In [0]:
# HANDLE MISSING OR INVALID VALUES: rating, book_id, review_text
from pyspark.sql import functions as F

# Normalize review_text for reliable checks (trim control chars)
df = df.withColumn(
    "review_text",
    F.trim(F.regexp_replace(F.col("review_text"), r"[\u0000-\u001F\u007F]", ""))
)

# Cast rating to int for validation
df = df.withColumn("rating", F.col("rating").cast("int"))

# Drop rows with:
# - rating null or outside [1..5]
# - book_id null or empty after trim
# - review_text null or empty after trim
df = df.filter(
    F.col("rating").isNotNull() &
    (F.col("rating").between(1, 5)) &
    F.col("book_id").isNotNull() &
    (F.length(F.trim(F.col("book_id"))) > 0) &
    F.col("review_text").isNotNull() &
    (F.length(F.trim(F.col("review_text"))) > 0)
)

df.select("rating", "book_id", "review_text").show(10, truncate=False)


+------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# CHECK THAT rating, book_id, review_text HAVE NO MISSING VALUES
from pyspark.sql import functions as F

missing_counts = (
    df.select(
        F.count(F.when(F.col("rating").isNull(), 1)).alias("missing_rating"),
        F.count(F.when(F.col("book_id").isNull() | (F.trim(F.col("book_id")) == ""), 1)).alias("missing_book_id"),
        F.count(F.when(F.col("review_text").isNull() | (F.trim(F.col("review_text")) == ""), 1)).alias("missing_review_text")
    )
)

missing_counts.show()


+--------------+---------------+-------------------+
|missing_rating|missing_book_id|missing_review_text|
+--------------+---------------+-------------------+
|             0|              0|                  0|
+--------------+---------------+-------------------+



In [0]:
# DROP REVIEWS WITH EXTREMELY SHORT review_text (<10 characters)
from pyspark.sql import functions as F

# Create review_length column (character count after trimming)
df = df.withColumn("review_length", F.length(F.trim(F.col("review_text"))))

# Filter out very short reviews (<10 chars)
df = df.filter(F.col("review_length") >= 10)

# check
df.select("review_length", "review_text").show(10, truncate=False)


+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# CHECK AND REMOVE INVALID OR FUTURE DATES IN date_added_iso
from pyspark.sql import functions as F

# Ensure date_added_iso is in DateType
df = df.withColumn("date_added_iso", F.col("date_added_iso").cast("date"))

# Current date for comparison
current_date = F.current_date()

# Keep only rows where date_added_iso is not null and not in the future
df = df.filter(
    F.col("date_added_iso").isNotNull() &
    (F.col("date_added_iso") <= current_date)
)

# verification
df.select("date_added_iso").summary("count", "min", "max").show()


+-------+
|summary|
+-------+
|  count|
|    min|
|    max|
+-------+



In [0]:
# 1. Inspect raw date strings
df.select("date_added_iso").show(10, truncate=False)

# 2. Verify type
df.printSchema()

# 3. If it's string, re-parse and cast properly
from pyspark.sql import functions as F
df = df.withColumn(
    "date_added_iso",
    F.to_date(F.col("date_added_iso"), "yyyy-MM-dd")
)

# 4. Re-run validation summary
df.select("date_added_iso").summary("count", "min", "max").show()


+--------------+
|date_added_iso|
+--------------+
|2014-03-06    |
|2012-10-01    |
|2014-08-30    |
|2012-11-27    |
|2016-09-30    |
|2015-02-23    |
|2017-01-10    |
|2010-02-08    |
|2014-11-26    |
|2012-01-22    |
+--------------+
only showing top 10 rows
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_added_parsed: timestamp (nullable = true)
 |-- date_added_iso: date (nullable = true)
 |-- review_length: integer (nullable = true)

+-------+
|summary|
+-------+
|  count|
|    min|
|    max|
+-------+



In [0]:
from pyspark.sql import functions as F

# Check for null or future dates
invalid_future = df.filter(
    (F.col("date_added_iso").isNull()) |
    (F.col("date_added_iso") > F.current_date())
)

print("Invalid or future dates:", invalid_future.count())

# display a few if any exist
invalid_future.select("date_added_iso").show(10, truncate=False)

# If count > 0, drop them
df = df.filter(
    F.col("date_added_iso").isNotNull() &
    (F.col("date_added_iso") <= F.current_date())
)

# Confirm remaining range
df.select(
    F.min("date_added_iso").alias("min_date"),
    F.max("date_added_iso").alias("max_date"),
    F.count("date_added_iso").alias("valid_count")
).show()


Invalid or future dates: 0
+--------------+
|date_added_iso|
+--------------+
+--------------+

+----------+----------+-----------+
|  min_date|  max_date|valid_count|
+----------+----------+-----------+
|1943-04-01|2017-11-04|   14971312|
+----------+----------+-----------+



In [0]:
# REPLACE MISSING n_votes WITH 0
from pyspark.sql import functions as F

df = df.withColumn("n_votes", F.when(F.col("n_votes").isNull(), 0).otherwise(F.col("n_votes")))

# verification
df.select(
    F.count(F.when(F.col("n_votes").isNull(), 1)).alias("missing_n_votes"),
    F.min("n_votes").alias("min_n_votes"),
    F.max("n_votes").alias("max_n_votes")
).show()


+---------------+-----------+-----------+
|missing_n_votes|min_n_votes|max_n_votes|
+---------------+-----------+-----------+
|              0|         -3|       3942|
+---------------+-----------+-----------+



In [0]:
# TRIM ALL TEXT COLUMNS AND CAPITALIZE EACH WORD FOR TITLE AND NAME
from pyspark.sql import functions as F

# Trim all text-based columns
text_cols = ["title", "name", "review_text", "language_code"]

for c in text_cols:
    if c in df.columns:
        df = df.withColumn(c, F.trim(F.col(c)))

# Capitalize each word for title and name
if "title" in df.columns:
    df = df.withColumn("title", F.initcap(F.col("title")))
if "name" in df.columns:
    df = df.withColumn("name", F.initcap(F.col("name")))

# verification
df.select("title", "name").show(10, truncate=False)


+---------------------------------------------------------+---------------+
|title                                                    |name           |
+---------------------------------------------------------+---------------+
|Blinded By Grace (cotillion Ball #5)                     |Becky Lower    |
|Timothy (timothy #1)                                     |Mark Tufo      |
|The Life And Legend Of Obi-wan Kenobi                    |Ryder Windham  |
|The Time Keeper                                          |Mitch Albom    |
|Sweet Christmas Kisses 3                                 |Mona Risk      |
|The Girl On The Train                                    |Paula Hawkins  |
|The Love Song                                            |Kay Simone     |
|Twenties Girl                                            |Sophie Kinsella|
|Whiskey, You're The Devil (an Addison Holmes Mystery, #4)|Liliana Hart   |
|Classified As Murder (cat In The Stacks, #2)             |Miranda James  |
+-----------

In [0]:
# AGGREGATIONS

from pyspark.sql import functions as F

# Ensure word count column
df = df.withColumn("word_count", F.size(F.split(F.col("review_text"), r"\s+")))

# 1) Average rating per BookID
avg_rating_per_book = (
    df.groupBy("book_id")
      .agg(F.avg("rating").alias("avg_rating_per_book"))
)

# 2) Number of reviews per BookID
n_reviews_per_book = (
    df.groupBy("book_id")
      .agg(F.count("review_id").alias("n_reviews_per_book"))
)

# 3) Average rating per AuthorName
avg_rating_per_author = (
    df.groupBy("name")
      .agg(F.avg("rating").alias("avg_rating_per_author"))
)

# 4) Word count statistics on reviews (global)
wordcount_stats_global = df.select(
    F.count("*").alias("n_rows"),
    F.min("word_count").alias("min_words"),
    F.expr("percentile_approx(word_count, 0.5)").alias("p50_words"),
    F.avg("word_count").alias("avg_words"),
    F.expr("percentile_approx(word_count, 0.9)").alias("p90_words"),
    F.max("word_count").alias("max_words")
)

# word count stats per BookID
wordcount_stats_per_book = (
    df.groupBy("book_id")
      .agg(
          F.min("word_count").alias("min_words"),
          F.expr("percentile_approx(word_count, 0.5)").alias("p50_words"),
          F.avg("word_count").alias("avg_words"),
          F.expr("percentile_approx(word_count, 0.9)").alias("p90_words"),
          F.max("word_count").alias("max_words")
      )
)

# join book-level aggregates into one table
book_aggregates = (
    avg_rating_per_book.alias("a")
    .join(n_reviews_per_book.alias("b"), "book_id", "inner")
)


In [0]:
# REMAINING ITEMS

from pyspark.sql import functions as F

# 1) Ensure author_id is present
if "author_id" in df.columns:
    df = df.filter(F.col("author_id").isNotNull() & (F.length(F.trim(F.col("author_id"))) > 0))

In [0]:
# 2) De-duplicate rows
#    a) by review_id when available
if "review_id" in df.columns:
    df = df.dropDuplicates(["review_id"])
#    b) also enforce uniqueness by (user_id, book_id) when both exist
if all(c in df.columns for c in ["user_id", "book_id"]):
    df = df.dropDuplicates(["user_id", "book_id"])



In [0]:
# 3) Normalize text fields not yet standardized
#    - keep Title/Name capitalization (already done)
#    - lower-case review_text and language_code; strip malformed control chars across all text columns
def strip_ctrl(col):
    return F.regexp_replace(F.col(col), r"[\u0000-\u001F\u007F]", "")

if "review_text" in df.columns:
    df = df.withColumn("review_text", F.lower(F.trim(strip_ctrl("review_text"))))
if "language_code" in df.columns:
    df = df.withColumn("language_code", F.lower(F.trim(strip_ctrl("language_code"))))



In [0]:
# 4) Numeric validations across all numeric columns (verify ranges; do not drop columns)
numeric_cols = [c for c, t in df.dtypes if t in ("int", "bigint", "double", "float", "decimal")]
checks = {}
for c in numeric_cols:
    stats = df.select(
        F.count(F.when(F.col(c).isNull(), 1)).alias("nulls"),
        F.min(F.col(c)).alias("min"),
        F.max(F.col(c)).alias("max")
    ).first()
    checks[c] = {"nulls": stats["nulls"], "min": stats["min"], "max": stats["max"]}

# Specific expected ranges
checks["rating_out_of_range"] = df.filter(~F.col("rating").between(1, 5)).count() if "rating" in df.columns else None
checks["n_votes_negative"] = df.filter(F.col("n_votes") < 0).count() if "n_votes" in df.columns else None

print(checks)

{'rating': {'nulls': 0, 'min': 1, 'max': 5}, 'n_votes': {'nulls': 0, 'min': -3, 'max': 3942}, 'review_length': {'nulls': 0, 'min': 10, 'max': 27210}, 'word_count': {'nulls': 0, 'min': 1, 'max': 5443}, 'rating_out_of_range': 0, 'n_votes_negative': 1523}


In [0]:
# FEATURE PREPARATION — WORD COUNT + BOOK-LEVEL AGGREGATES

from pyspark.sql import functions as F

# 1) Compute review length in words
df = df.withColumn("review_length_words", F.size(F.split(F.col("review_text"), r"\s+")))

# 2) Aggregate by book_id: average rating and number of reviews
book_aggs = (
    df.groupBy("book_id")
      .agg(
          F.avg("rating").alias("avg_rating_per_book"),
          F.count("review_id").alias("n_reviews_per_book")
      )
)

# 3) Join aggregates back to the main dataframe (preserve all original columns)
df_features = (
    df.join(book_aggs, on="book_id", how="left")
)

# verification
df_features.select(
    "book_id", "avg_rating_per_book", "n_reviews_per_book", "review_length_words"
).show(10, truncate=False)


+--------+-------------------+------------------+-------------------+
|book_id |avg_rating_per_book|n_reviews_per_book|review_length_words|
+--------+-------------------+------------------+-------------------+
|27189194|4.241206030150754  |398               |323                |
|18668056|3.954337899543379  |219               |288                |
|6900    |4.1833941605839415 |2192              |7                  |
|6186452 |4.059360730593608  |438               |1057               |
|2696    |3.57703081232493   |357               |13                 |
|1241    |3.3550863723608444 |1042              |15                 |
|33572350|3.789237668161435  |223               |76                 |
|25721421|3.6842105263157894 |19                |208                |
|7417780 |4.431226765799257  |269               |227                |
|792161  |3.9927884615384617 |416               |453                |
+--------+-------------------+------------------+-------------------+
only showing top 10 

In [0]:
# SAVE CLEANED AND ENRICHED DATASET TO GOLD LAYER AS DELTA features_v1

# Write DataFrame to Delta format under gold/features_v1
df_features.write.mode("overwrite").format("delta").save(f"{gold_path}/features_v1")

# verification
from pyspark.sql import functions as F
features_check = spark.read.format("delta").load(f"{gold_path}/features_v1")
features_check.select(
    F.count("*").alias("rows"),
    F.countDistinct("book_id").alias("unique_books"),
    F.countDistinct("author_id").alias("unique_authors")
).show()


+--------+------------+--------------+
|    rows|unique_books|unique_authors|
+--------+------------+--------------+
|14970894|     2010165|        438020|
+--------+------------+--------------+



### Verification Results of Gold Layer (`features_v1`)

#### Interpretation
- That output is correct and expected not identical to original row count.
- During cleaning, we removed duplicates and dropped invalid or missing fields, and filtered short reviews.
- Those actions naturally reduce the total number of rows.
- The distinct counts of `book_id` and `author_id` remain high because each book or author can still have multiple valid reviews.
- These indicate the dataset was deduplicated and cleaned successfully while maintaining proper relational diversity.

In [0]:
# VERIFY OUTPUT FROM GOLD LAYER (features_v1)

from pyspark.sql import functions as F

# Reload the saved dataset
features = spark.read.format("delta").load(f"{gold_path}/features_v1")

# Inspect schema
features.printSchema()

# Record count
print("Total records:", features.count())

# Show sample rows
features.show(10, truncate=False)

# Optional: sanity checks for new features
features.select(
    F.min("review_length_words").alias("min_length"),
    F.avg("review_length_words").alias("avg_length"),
    F.max("review_length_words").alias("max_length"),
    F.min("avg_rating_per_book").alias("min_avg_rating"),
    F.max("avg_rating_per_book").alias("max_avg_rating")
).show()

root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_added_parsed: timestamp (nullable = true)
 |-- date_added_iso: date (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- word_count: integer (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating_per_book: double (nullable = true)
 |-- n_reviews_per_book: long (nullable = true)

Total records: 14970894
+--------+--------------------------------+-----------------------------------------------------------------------------------+---------+----------------+--------------------------------+-----